# Example
this file show the process of RAG for a speech-to-text uses case

in this notebook:

1. **Speech-to-text (STT)**: we perform STT on a iven audio file, using a Whisper open-source model

2. *Chunking*: we take the extracted text, we create a **Langchain Document** from it, then we split the document into **chunks** using the spacy library.

3. **Summarization**: we take the chunks and summarize them into a single big summary using **map reduce** approach: see https://python.langchain.com/docs/use_cases/summarization

4. **Embedding and store**: we create **embeddings** of the summary and then store the embeddins on **Qdrant DB**

Note: this approach will be used for all of our use cases, not only for Speech-to-text!

In [1]:
import whisper
import sys
sys.path.append('..')
from utils import AWSTexttract, TextSplitter, LangChainAI, QDrantDBManager, EmbeddingFunction
import pytube
model = whisper.load_model("base")
import os

# 1. Speech-to-text
Im using Whisper model her ebut feel free to use other open source models if you want 

In [2]:
# model = whisper.load_model("large")
model = whisper.load_model("base")

In [3]:
filename=r"C:\Users\ELAFACRB1\Downloads\AUD-20240319-WA0006.m4a" #put your file here 
text = model.transcribe(filename)
text['text']

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


" Ciao ragazzi buongiorno. Buongiorno a tutti. Si sente bene, però anche dietro si sente. Allora ragazzi un paio di avvisimo, nevo farvi. Per quanto riguarda il mese di marzo di affile, troverete quando pubblica in un paio di appuntamenti in più, di approfondimenti, oltre le canoniche, lezioni che pensate. E saranno due approfondimenti due focos, uno di penale e uno in materia di civile. Uno sarà registrato, quindi veloca richeremo nei cruxcotti e rimarrà vostra disposizione da parte della proessore ssamorello. E poi l'altro invece sarà indiretta un paio di ore su piattaforma zoom con l'avvocato del lo stato in manu e le manzo. Invece per quanto riguarda l'altra novità è che da questo l'unedice, da l'unedì prossimo, torna l'appuntamento ogni l'unedì diciamo di fine mese. Per gli approfondimenti con il presidente, sempre su piattaforma zoom ovviamente, gli approfondimenti su tutte le questioni di rilievo ovviamente che avete trattato nel mese appena concluso. Quindi magari il presidente

# 2. Chunking

In [9]:
text_summarizer = TextSplitter(chunk_size=16000, chunk_overlap=0)

Text splitting

In [12]:
#this function uses semantic text splitting using spacy 
#other functions uses 
splitted_text = text_summarizer.semantic_split_text(text['text']) 
len(splitted_text)

26

Creation of Langchain documents

In [13]:
docs =  text_summarizer.create_langchain_documents(splitted_text, {"source": "mp3"})
len(docs)

26

# 3. Summarization

In [16]:
langchain_client = LangChainAI()
summary=langchain_client.summarize_text(docs)

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


# 4. Embedding and store

Indexing on Qdrant vector database

In [ ]:
QDRANT_URL=os.getenv('QDRANT_URL')
COLLECTION_NAME=os.getenv('COLLECTION_NAME')

#here Im using the EmbeddingFunction class to instantiate an embedding model 
#(openai in this case, feel free to test other embedding functions in the future)

#as a record manager (database used to store hash of the embedded documents used to avoid storing the 
#same document twice) I'm using a local sql database, we can use other db in the future

qdrantClient = QDrantDBManager(
    url=QDRANT_URL,
    port=6333,
    collection_name=COLLECTION_NAME,
    vector_size=1536,
    embedding=EmbeddingFunction('openAI').embedder,
    record_manager_url="sqlite:///record_manager_cache.sql"
)
qdrantClient.index_documents(docs)